In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [2]:
conf = SparkConf().setMaster("local[*]").setAppName("Iris_Project")
sc = SparkContext.getOrCreate(conf = conf)
ss =  SparkSession.builder.config(conf=conf)

In [3]:
#Load Iris data
rawdata = sc.textFile("G:\SparkProject\Iris data\Iris Data\iris.csv")

In [4]:
#rawdata.take(3)

In [5]:
tmp_irisdata = sc.parallelize(rawdata.first())

In [6]:
irisNohead = rawdata.subtract(tmp_irisdata)

In [7]:
#irisNohead.take(3)

In [8]:
from pyspark.sql import Row

In [9]:
tmp_irisNohead = irisNohead.map(lambda x: x.split(','))

In [10]:
tmp_irisNohead.take(3)

[['5.1', '3.5', '1.4', '.2', '"Setosa"'],
 ['5', '3.4', '1.5', '.2', '"Setosa"'],
 ['5.4', '3.7', '1.5', '.2', '"Setosa"']]

In [11]:
from pyspark.sql import Row
#Create a Data Frame from the data
irisMap = tmp_irisNohead.map(lambda p: Row(SEPAL_LENGTH=float(p[0]),\
                                SEPAL_WIDTH=float(p[1]), \
                                PETAL_LENGTH=float(p[2]), \
                                PETAL_WIDTH=float(p[3]), \
                                SPECIES=p[4] ))

In [12]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc) #only SQL.Context has createDataFrame

In [13]:
#Infer the schema and register the DataFrame as a table
irisDf = sqlContext.createDataFrame(irisMap)
irisDf.cache()

DataFrame[PETAL_LENGTH: double, PETAL_WIDTH: double, SEPAL_LENGTH: double, SEPAL_WIDTH: double, SPECIES: string]

In [14]:
irisDf = irisDf.withColumn("SEPAL_LENGTH", irisDf.SEPAL_LENGTH.cast('double'))
irisDf = irisDf.withColumn("SEPAL_WIDTH", irisDf.SEPAL_WIDTH.cast('double'))
irisDf = irisDf.withColumn("PETAL_LENGTH", irisDf.PETAL_LENGTH.cast('double'))
irisDf = irisDf.withColumn("PETAL_WIDTH", irisDf.PETAL_WIDTH.cast('double'))

In [15]:
irisDf.cache()

DataFrame[PETAL_LENGTH: double, PETAL_WIDTH: double, SEPAL_LENGTH: double, SEPAL_WIDTH: double, SPECIES: string]

In [16]:
#Create indexer for training and test sets

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

species_indexer = StringIndexer(inputCol="SPECIES", outputCol="ID_SPEC")
species_encoder = OneHotEncoder(inputCol="ID_SPEC", outputCol="SPEC_FACT")

In [17]:
#Make a VectorAssembler
vec_assembler = VectorAssembler(inputCols=["SEPAL_LENGTH","SEPAL_WIDTH",
                                          "PETAL_LENGTH","PETAL_WIDTH"], outputCol="fetures")


In [18]:
#Import pipeline

from pyspark.ml import Pipeline

In [19]:
#Make the Pipeline
iris_pipe = Pipeline(stages=[species_indexer, species_encoder, vec_assembler])

In [20]:
#Remove missing values
#irisDf = irisDf.filter("SEPAL_LENGTH is not NULL and SEPAL_WIDTH is not NULL and PETAL_LENGTH is not NULL and PETAL_WIDTH is not NULL")

In [21]:
si_model = species_indexer.fit(irisDf)

Py4JJavaError: An error occurred while calling o141.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 6.0 failed 1 times, most recent failure: Lost task 1.0 in stage 6.0 (TID 10, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "D:\Spark\Spark\python\lib\pyspark.zip\pyspark\worker.py", line 177, in main
  File "D:\Spark\Spark\python\lib\pyspark.zip\pyspark\worker.py", line 172, in process
  File "D:\Spark\Spark\python\lib\pyspark.zip\pyspark\serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-11-3a999b631149>", line 3, in <lambda>
ValueError: could not convert string to float: '"sepal.length"'

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.PythonRunner$$anon$1.<init>(PythonRDD.scala:234)
	at org.apache.spark.api.python.PythonRunner.compute(PythonRDD.sca

In [ ]:
print(type(irisDf))
irisDf.take(3)